# **Imports:**

----

In [36]:
from requests import get
from bs4 import BeautifulSoup
import os
import requests
import pandas as pd
import unicodedata
import re
import json

import matplotlib.pyplot as plt

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords


from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
import acquire
import prepare

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

-----

# **Acquire:**

In [37]:
#utilizing json file created from acquire.py, using pandas read json to create a dataframe
df = pd.read_json('data2.json')

In [38]:
df.head()

,repo,language,readme_contents
0,IrisShaders/Iris,Java,![Iris: An open-source shaders mod compatible ...
1,returntocorp/semgrep,Java,"</br>\n<p align=""center"">\n <a href=""https:..."
2,CaffeineMC/sodium-fabric,Java,"<img src=""src/main/resources/assets/sodium/ico..."
3,keycloak/keycloak,Java,# Keycloak\n\nKeycloak is an Open Source Ident...
4,NationalSecurityAgency/ghidra,Java,"<img src=""Ghidra/Features/Base/src/main/resour..."


In [39]:
#initial is 225 records x 3 fields, all are objects.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             225 non-null    object
 1   language         225 non-null    object
 2   readme_contents  225 non-null    object
dtypes: object(3)
memory usage: 5.4+ KB


------

To Do's:

1.) drop duplicates, drop nans

2.) lower case language and repo columns, and readme_contents as well.

3.) drop non alpha-numeric values.

4.) Drop words which are excessively long, appears there is combination of words



------

# **Prepare:**

In [40]:
#utilizing prepare function which will retrun a df containing original columns and columns which have been normalized, lemmatized, tokenized and had stopwords removed or added.
df = prepare.prep_repo_data(df)

In [41]:
#looking at the initial and new columns.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 198
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   repo_name                174 non-null    object
 1   language                 174 non-null    object
 2   readme_contents          174 non-null    object
 3   repo_name_cleaned        174 non-null    object
 4   language_cleaned         174 non-null    object
 5   readme_contents_cleaned  174 non-null    object
dtypes: object(6)
memory usage: 9.5+ KB


In [42]:
df.shape

(174, 6)

In [43]:
df.head(10)

,repo_name,language,readme_contents,repo_name_cleaned,language_cleaned,readme_contents_cleaned
0,IrisShaders/Iris,Java,![Iris: An open-source shaders mod compatible ...,irisshadersiris,java,iris opensource shaders mod compatible optifin...
1,returntocorp/semgrep,Java,"</br>\n<p align=""center"">\n <a href=""https:...",returntocorpsemgrep,java,br p aligncenter hrefhttpssemgrepdevimg srchtt...
2,CaffeineMC/sodium-fabric,Java,"<img src=""src/main/resources/assets/sodium/ico...",caffeinemcsodiumfabric,java,img srcsrcmainresourcesassetssodiumiconpng wid...
3,keycloak/keycloak,Java,# Keycloak\n\nKeycloak is an Open Source Ident...,keycloakkeycloak,java,keycloak keycloak open source identity access ...
4,NationalSecurityAgency/ghidra,Java,"<img src=""Ghidra/Features/Base/src/main/resour...",nationalsecurityagencyghidra,java,img srcghidrafeaturesbasesrcmainresourcesimage...
5,alibaba/Sentinel,Java,"<img src=""https://user-images.githubuserconten...",alibabasentinel,java,img srchttpsuserimagesgithubusercontentcom9434...
6,square/okhttp,Java,OkHttp\n======\n\nSee the [project website][ok...,squareokhttp,java,okhttp see project websiteokhttp documentation...
7,alibaba/nacos,Java,"\n<img src=""doc/Nacos_Logo.png"" width=""50%"" sy...",alibabanacos,java,img srcdocnacos_logopng width50 syt height50 n...
8,opensearch-project/OpenSearch,Java,"<img src=""https://opensearch.org/assets/brand/...",opensearchprojectopensearch,java,img srchttpsopensearchorgassetsbrandsvglogoope...
9,sqshq/piggymetrics,Java,[![Build Status](https://travis-ci.org/sqshq/P...,sqshqpiggymetrics,java,build statushttpstravisciorgsqshqpiggymetricss...


In [44]:
java_read_df = df[df.language_cleaned == 'java']


,repo_name,language,readme_contents,repo_name_cleaned,language_cleaned,readme_contents_cleaned
0,IrisShaders/Iris,Java,![Iris: An open-source shaders mod compatible ...,irisshadersiris,java,iris opensource shaders mod compatible optifin...
1,returntocorp/semgrep,Java,"</br>\n<p align=""center"">\n <a href=""https:...",returntocorpsemgrep,java,br p aligncenter hrefhttpssemgrepdevimg srchtt...
2,CaffeineMC/sodium-fabric,Java,"<img src=""src/main/resources/assets/sodium/ico...",caffeinemcsodiumfabric,java,img srcsrcmainresourcesassetssodiumiconpng wid...
3,keycloak/keycloak,Java,# Keycloak\n\nKeycloak is an Open Source Ident...,keycloakkeycloak,java,keycloak keycloak open source identity access ...
4,NationalSecurityAgency/ghidra,Java,"<img src=""Ghidra/Features/Base/src/main/resour...",nationalsecurityagencyghidra,java,img srcghidrafeaturesbasesrcmainresourcesimage...
5,alibaba/Sentinel,Java,"<img src=""https://user-images.githubuserconten...",alibabasentinel,java,img srchttpsuserimagesgithubusercontentcom9434...
6,square/okhttp,Java,OkHttp\n======\n\nSee the [project website][ok...,squareokhttp,java,okhttp see project websiteokhttp documentation...
7,alibaba/nacos,Java,"\n<img src=""doc/Nacos_Logo.png"" width=""50%"" sy...",alibabanacos,java,img srcdocnacos_logopng width50 syt height50 n...
8,opensearch-project/OpenSearch,Java,"<img src=""https://opensearch.org/assets/brand/...",opensearchprojectopensearch,java,img srchttpsopensearchorgassetsbrandsvglogoope...
9,sqshq/piggymetrics,Java,[![Build Status](https://travis-ci.org/sqshq/P...,sqshqpiggymetrics,java,build statushttpstravisciorgsqshqpiggymetricss...


In [59]:
df['readme_contents_cleaned'] = df['readme_contents_cleaned'].str.findall('\w{1,12}').str.join(' ')


,repo_name,language,readme_contents,repo_name_cleaned,language_cleaned,readme_contents_cleaned
0,IrisShaders/Iris,Java,![Iris: An open-source shaders mod compatible ...,irisshadersiris,java,iris opensource shaders mod compatible optifin...
1,returntocorp/semgrep,Java,"</br>\n<p align=""center"">\n <a href=""https:...",returntocorpsemgrep,java,br aligncenter hrefhttpssem gre pdevimg srchtt...
2,CaffeineMC/sodium-fabric,Java,"<img src=""src/main/resources/assets/sodium/ico...",caffeinemcsodiumfabric,java,img srcsrcmainre sou rcesassetsso diu miconpng...
3,keycloak/keycloak,Java,# Keycloak\n\nKeycloak is an Open Source Ident...,keycloakkeycloak,java,keycloak keycloak open source identity access ...
4,NationalSecurityAgency/ghidra,Java,"<img src=""Ghidra/Features/Base/src/main/resour...",nationalsecurityagencyghidra,java,img srcghidrafea tur esbasesrcmai nre sourcesi...


In [61]:
def lang_split(df):
    '''
    This function take in the readme data acquired
    performs a split and stratifies language_cleaned column.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=245, 
                                        stratify=df.language_cleaned)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=245, 
                                   stratify=train_validate.language_cleaned)
    return train, validate, test

In [64]:
#splitting the data into a train, validate and test set
train, validate, test = lang_split(df)

## df.copy for altering 

-----

# **Exploratory**:

In [65]:
java_read_df = train[train.language_cleaned == 'java']
python_read_df = train[train.language_cleaned == 'python']
r_read_df = train[train.language_cleaned == 'r']
javascript_read_df = train[train.language_cleaned == 'javascript']